In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
torch.manual_seed(42)

In [3]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [4]:
data=pd.read_excel('/content/Occupancy detection.xlsx')
data.head()

,id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
1,2,2015-02-02 14:19:00,23.7180,26.290,578.400000,760.400000,0.004773,1
2,3,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
3,4,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
4,5,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


In [5]:
data.isnull().sum()

,0
id,0
date,0
Temperature,0
Humidity,0
Light,0
CO2,0
HumidityRatio,0
Occupancy,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20560 entries, 0 to 20559
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             20560 non-null  int64         
 1   date           20560 non-null  datetime64[ns]
 2   Temperature    20560 non-null  float64       
 3   Humidity       20560 non-null  float64       
 4   Light          20560 non-null  float64       
 5   CO2            20560 non-null  float64       
 6   HumidityRatio  20560 non-null  float64       
 7   Occupancy      20560 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 1.3 MB


In [7]:
X=data.drop(['Occupancy'],axis=1)
y=data['Occupancy']

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
#creating custom class
class customDataset(Dataset):
  def __init__(self,features,labels):
    # Convert 'date' column to numerical representation before converting to tensor
    features['date'] = pd.to_numeric(features['date'])
    self.features=torch.tensor(features.values.astype(np.float32),dtype=torch.float32)
    self.labels=torch.tensor(labels.values,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]


In [10]:
#creating training and testing object
train_dataset=customDataset(X_train,y_train)
test_dataset=customDataset(X_test,y_test)


In [11]:
#creating train and test data loader
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False,pin_memory=True)

In [12]:
len(train_loader)

257

In [13]:
len(test_loader)

65

In [14]:
#custom nn model
class MyNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(num_features,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,2)
    )

  def forward(self,x):
    return self.model(x)

In [15]:
learning_rate=0.01
epochs=25

In [16]:
#instatiate the model
model=MyNN(num_features=X_train.shape[1])
model=model.to(device)
#loss
criterion=nn.CrossEntropyLoss()
#optimizer
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [17]:
#training loop
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    #forwrd pass
    output=model(batch_features)

    #loss
    loss=criterion(output,batch_labels)

    #optimizer
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    total_epoch_loss+=loss.item()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


Epoch 1/25, Loss: 0.4587242901325226
Epoch 2/25, Loss: 0.5820340514183044
Epoch 3/25, Loss: 0.5992434024810791
Epoch 4/25, Loss: 0.489163339138031
Epoch 5/25, Loss: 0.6364966034889221
Epoch 6/25, Loss: 0.4876343607902527
Epoch 7/25, Loss: 0.5445106625556946
Epoch 8/25, Loss: 0.526238203048706
Epoch 9/25, Loss: 0.5263947248458862
Epoch 10/25, Loss: 0.5630150437355042
Epoch 11/25, Loss: 0.6374027132987976
Epoch 12/25, Loss: 0.5807225704193115
Epoch 13/25, Loss: 0.4879825711250305
Epoch 14/25, Loss: 0.5263910293579102
Epoch 15/25, Loss: 0.5255116820335388
Epoch 16/25, Loss: 0.6363440752029419
Epoch 17/25, Loss: 0.5445159673690796
Epoch 18/25, Loss: 0.6138518452644348
Epoch 19/25, Loss: 0.4703322649002075
Epoch 20/25, Loss: 0.42564278841018677
Epoch 21/25, Loss: 0.5064784288406372
Epoch 22/25, Loss: 0.4867016673088074
Epoch 23/25, Loss: 0.48692992329597473
Epoch 24/25, Loss: 0.5817915201187134
Epoch 25/25, Loss: 0.4673749506473541


In [18]:
#evaluate
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=7, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [19]:
#evaluation
#total number of samples
total=0
#total number of correct samples
correct=0

with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    #forward pass
    outputs=model(batch_features)

    _,predicted=torch.max(outputs,1)

    total+=batch_labels.size(0)
    correct+=(predicted==batch_labels).sum().item()

  print(100*(correct/total))


77.99124513618678


In [20]:
#evaluation for train loader

total=0
correct=0

with torch.no_grad():
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    outputs=model(batch_features)

    _,predicted=torch.max(outputs,1)

    total+=batch_labels.size(0)

    correct=(predicted==batch_labels).sum().item()

  print(100*(correct/total))

0.25535019455252916
